In [21]:
import pandas as pd
import xml.etree.ElementTree as ET
from urllib.request import urlopen
from tqdm import trange
from bs4 import BeautifulSoup as bs
import re

In [22]:
#기존의 크롤링 방법 말고 다른 방식을 써보자

id='mugudeli601'
url = "https://www.law.go.kr/DRF/lawSearch.do?OC={}&target=prec&type=XML".format(id)
response = urlopen(url).read()
bs(response)

c:\Users\Heeyo\AppData\Local\Programs\Python\Python39\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


<?xml version="1.0" encoding="UTF-8"?><html><body><precsearch><target>prec</target>&lt;키워드&gt;*키워드&gt;<section>evtNm</section><totalcnt>85083</totalcnt><page>1</page><prec id="1">&lt;판례일련번호&gt;230881판례일련번호&gt;&lt;사건명&gt;&lt;![CDATA[소유권이전등기]]&gt;사건명&gt;&lt;사건번호&gt;2021다216766사건번호&gt;&lt;선고일자&gt;2022.08.31선고일자&gt;&lt;법원명&gt;대법원법원명&gt;&lt;법원종류코드&gt;법원종류코드&gt;&lt;사건종류명&gt;민사사건종류명&gt;&lt;사건종류코드&gt;400101사건종류코드&gt;&lt;판결유형&gt;판결판결유형&gt;&lt;선고&gt;선고선고&gt;&lt;판례상세링크&gt;/DRF/lawService.do?OC=mugudeli601&amp;target=prec&amp;ID=230881&amp;type=HTML&amp;mobileYn=판례상세링크&gt;</prec><prec id="2">&lt;판례일련번호&gt;230867판례일련번호&gt;&lt;사건명&gt;&lt;![CDATA[보험금]]&gt;사건명&gt;&lt;사건번호&gt;2018다304014사건번호&gt;&lt;선고일자&gt;2022.08.31선고일자&gt;&lt;법원명&gt;대법원법원명&gt;&lt;법원종류코드&gt;법원종류코드&gt;&lt;사건종류명&gt;민사사건종류명&gt;&lt;사건종류코드&gt;400101사건종류코드&gt;&lt;판결유형&gt;판결판결유형&gt;&lt;선고&gt;선고선고&gt;&lt;판례상세링크&gt;/DRF/lawService.do?OC=mugudeli601&amp;target=prec&amp;ID=230867&amp;type=HTML&amp;mobileYn=판례상세링크&gt;</prec><prec id="3">&lt;판례일련번

In [23]:
xtree = ET.fromstring(response) #response에 저장된 string으로부터 tree를 생성한다

In [24]:
#first check what is in it
'''for child in xtree:
    print(':',child.tag,child.attrib)
print(xtree.find('page').text)'''

"for child in xtree:\n    print(':',child.tag,child.attrib)\nprint(xtree.find('page').text)"

In [25]:
xtree[3].text #totalCnt

'85083'

In [26]:
for node in xtree[5:]:
    print('\n\n',node.attrib)
    tags=[]
    for child in node:
        tags.append(child.tag)
    for t in tags:
        print(node.find(t).text)



 {'id': '1'}
230881
소유권이전등기
2021다216766
2022.08.31
대법원
None
민사
400101
판결
선고
/DRF/lawService.do?OC=mugudeli601&target=prec&ID=230881&type=HTML&mobileYn=


 {'id': '2'}
230867
보험금
2018다304014
2022.08.31
대법원
None
민사
400101
판결
선고
/DRF/lawService.do?OC=mugudeli601&target=prec&ID=230867&type=HTML&mobileYn=


 {'id': '3'}
230871
통신비밀보호법위반
2020도1007
2022.08.31
대법원
None
형사
400102
판결
선고
/DRF/lawService.do?OC=mugudeli601&target=prec&ID=230871&type=HTML&mobileYn=


 {'id': '4'}
230873
부가가치세경정거부처분취소
2017두53170
2022.08.31
대법원
None
세무
400108
판결
선고
/DRF/lawService.do?OC=mugudeli601&target=prec&ID=230873&type=HTML&mobileYn=


 {'id': '5'}
230869
구상금
2018다212740
2022.08.31
대법원
None
민사
400101
판결
선고
/DRF/lawService.do?OC=mugudeli601&target=prec&ID=230869&type=HTML&mobileYn=


 {'id': '6'}
230877
모욕
2019도7370
2022.08.31
대법원
None
형사
400102
판결
선고
/DRF/lawService.do?OC=mugudeli601&target=prec&ID=230877&type=HTML&mobileYn=


 {'id': '7'}
230879
건축주명의변경절차이행청구의소
2019다282050
2022.08.31
대법원
None
민사
400101
판결
선고


In [27]:
totalCnt = int(xtree.find('totalCnt').text)#판례 페이지에 totalCnt라는 요소가 존재하는 것 같다. 
#왜 20개밖에 없나 했는데 원래 한 페이지당 20개라고 한다. 
#페이지마다 loop을 돌아가야 하는 것

In [28]:
#판례 목록 만들기 
cnt=0
page = 1
rows = []
for i in trange(int(totalCnt / 20)): #기다리는 바를 만드는 tqdm의 range 함수인 trange
    if i>1:
        break
    try:
        items = xtree[5:]
    except:
        break
        
    for node in items:
        data={}
        tags=[]
        for child in node:
            tags.append(child.tag)
        for t in tags:
            data[t]=re.sub(',','$',str(node.find(t).text))

        rows.append(data)
    cnt += 1
    url = "https://www.law.go.kr/DRF/lawSearch.do?OC=mugudeli601&target=prec&type=XML&page={}".format(page)
    response = urlopen(url).read()
    xtree = ET.fromstring(response)
    page+=1

  0%|          | 2/4254 [00:00<21:38,  3.28it/s]


In [29]:
#rows에 모든 정보가 담겨있음
prec_list=pd.DataFrame(rows,columns=rows[0].keys())
prec_list.head(5)

,판례일련번호,사건명,사건번호,선고일자,법원명,법원종류코드,사건종류명,사건종류코드,판결유형,선고,판례상세링크
0,230881,소유권이전등기,2021다216766,2022.08.31,대법원,None,민사,400101,판결,선고,/DRF/lawService.do?OC=mugudeli601&target=prec&...
1,230867,보험금,2018다304014,2022.08.31,대법원,None,민사,400101,판결,선고,/DRF/lawService.do?OC=mugudeli601&target=prec&...
2,230871,통신비밀보호법위반,2020도1007,2022.08.31,대법원,None,형사,400102,판결,선고,/DRF/lawService.do?OC=mugudeli601&target=prec&...
3,230873,부가가치세경정거부처분취소,2017두53170,2022.08.31,대법원,None,세무,400108,판결,선고,/DRF/lawService.do?OC=mugudeli601&target=prec&...
4,230869,구상금,2018다212740,2022.08.31,대법원,None,민사,400101,판결,선고,/DRF/lawService.do?OC=mugudeli601&target=prec&...


In [30]:
prec_list.columns

Index(['판례일련번호', '사건명', '사건번호', '선고일자', '법원명', '법원종류코드', '사건종류명', '사건종류코드',
       '판결유형', '선고', '판례상세링크'],
      dtype='object')

In [31]:
coluumns_new={'판례일련번호':'datanumber', '사건명':'title', '사건번호':'casenumber', '선고일자':'date', '법원명':'court', '법원종류코드':'courttype', '사건종류명':'casetypename', '사건종류코드':'casecode',
       '판결유형':'verdicttype', '선고':'sentence', '판례상세링크':'detaillink'}
prec_list=prec_list.rename(columns=coluumns_new)

In [32]:
#prec_list를 traverse하며 모든 판례정보를 긁기
import re

contents = ['사건명', '사건번호', '선고일자','판결유형','판시사항', '판결요지', '참조조문', '참조판례', '판례내용']
contents_rename=['title','casenumber','date','verdicttype','holding','summary','jmreference','prreference','prny']
rename_dict={a:b for a,b in zip(contents,contents_rename)}

In [37]:
rename_dict

{'사건명': 'title',
 '사건번호': 'casenumber',
 '선고일자': 'date',
 '판결유형': 'verdicttype',
 '판시사항': 'holding',
 '판결요지': 'summary',
 '참조조문': 'jmreference',
 '참조판례': 'prreference',
 '판례내용': 'prny'}

In [39]:
prec_list.columns

Index(['datanumber', 'title', 'casenumber', 'date', 'court', 'courttype',
       'casetypename', 'casecode', 'verdicttype', 'sentence', 'detaillink'],
      dtype='object')

In [44]:


def remove_tag(content):
    cleaned=re.sub(',','$',re.sub('<.*?>', '', content))
    return cleaned

case_list=[]
for pid,l in zip(prec_list['datanumber'],prec_list['detaillink']):
    url="https://www.law.go.kr"+l.replace('HTML','XML')
    response=urlopen(url).read()
    xtree=ET.fromstring(response)
    row=[]
    row.append(pid)
    for content in contents:
        try:
            text=xtree.find(content).text
            text=remove_tag(text)
        except:
            text="-"
        row.append(text)
    case_list.append(row)
    
case_list=pd.DataFrame(case_list,columns=['datanumber']+contents)
case_list.reset_index(inplace=True)
case_list=case_list.rename(columns=rename_dict)        

In [34]:
len(case_list)

40

In [35]:
from pathlib import Path  
filepath = Path('D:\\PanryeAI\\case_list.csv')  
case_list.to_csv(filepath,index=False)

In [45]:
case_list.head(10)

,index,datanumber,title,casenumber,date,verdicttype,holding,summary,jmreference,prreference,prny
0,0,230881,소유권이전등기,2021다216766,20220831,판결,[1] 1975. 12. 31. 지적법 개정 전에 복구된 구 토지대장이나 분배농...,[1] 1975. 12. 31. 지적법 개정 전에 복구된 구 토지대장상의 소유자...,[1] 민법 제186조$ 민사소송법 제202조$ 구 지적법(1975. 12. 31...,[1] 대법원 2013. 6. 27. 선고 2012다91354 판결$ 대법원 20...,【원고$ 피상고인】 원고【피 고】 피고【피고보조참가인$ 상고인】 피고보조참가인...
1,1,230867,보험금,2018다304014,20220831,판결,甲 자산운용회사가 乙 보험회사와 체결한 자산운용전문인 배상책임보험계약의 계약서 ...,甲 자산운용회사가 乙 보험회사와 체결한 자산운용전문인 배상책임보험계약의 계약서 ...,민법 제105조,-,【원고$ 피상고인】 칸서스자산운용 주식회사 (소송대리인 법무법인(유한) 지평 담당변...
2,2,230871,통신비밀보호법위반,2020도1007,20220831,판결,통신비밀보호법 제14조 제1항의 금지를 위반하는 행위는 같은 법 제3조 제1항 위반...,통신비밀보호법은 공개되지 않은 타인 간의 대화에 관하여 다음과 같이 정하고 있다...,통신비밀보호법 제3조 제1항$ 제14조 제1항$ 제16조 제1항,대법원 2006. 10. 12. 선고 2006도4981 판결(공2006하$ 1939...,【피 고 인】 피고인【상 고 인】 피고인【원심판결】 부산고법 2020. 1. 9. ...
3,3,230873,부가가치세경정거부처분취소,2017두53170,20220831,판결,어떠한 금액이 부가가치세의 과세표준에 포함되지 않는 에누리액에 해당하기 위해서는...,구 부가가치세법(2013. 6. 7. 법률 제11873호로 전부 개정되기 전의 것)...,구 부가가치세법(2013. 6. 7. 법률 제11873호로 전부 개정되기 전의 것)...,-,【원고$ 상고인】 에스케이텔레콤 주식회사 (소송대리인 법무법인(유한) 태평양 외 2...
4,4,230869,구상금,2018다212740,20220831,판결,[1] 자동차상해보험계약에 따른 보험금을 지급한 보험자가 보험자대위를 할 수 있...,[1] 자동차상해보험은 그 성질상 상해보험에 속하므로$ 자동차상해보험계약에 따른...,[1] 상법 제729조 / [2] 상법 제729조,[1] 대법원 2004. 7. 9. 선고 2003다29463 판결(공2004하$ ...,【원고$ 상고인 겸 피상고인】 삼성화재해상보험 주식회사 (소송대리인 법무법인(유한)...
5,5,230877,모욕,2019도7370,20220831,판결,[1] 모욕죄의 보호법익(=외부적 명예) 및 ‘모욕’의 의미 / 어떠한 표현이 ...,[1] 형법 제311조 모욕죄는 사람의 인격적 가치에 대한 사회적 평가를 의미하...,[1] 헌법 제21조$ 형법 제311조 / [2] 헌법 제21조$ 형법 제311조,[1] 대법원 2018. 11. 29. 선고 2017도2661 판결(공2019상$...,【피 고 인】 피고인【상 고 인】 피고인【원심판결】 수원지법 2019. 5. 3. ...
6,6,230879,건축주명의변경절차이행청구의소,2019다282050,20220831,판결,건축허가 또는 신고에 관한 건축주 명의가 수인으로 되어 있을 경우$ 공동건축주 ...,행정청으로부터 허가를 받거나 행정청에 신고(이하 이러한 허가와 신고를 합하여 ‘...,민법 제265조,대법원 2015. 9. 10. 선고 2012다23863 판결(공2015하$ 1457),【원고$ 상고인】 대한예수교장로회 ○○교회 (소송대리인 법무법인 다비다 담당변호사 ...
7,7,230859,손해배상(기)[긴급조치 제9호위반 혐의로 수사 및 유죄판결을 받은 사람들 또는 그 ...,2018다212610,20220830,전원합의체 판결,구 국가안전과 공공질서의 수호를 위한 대통령긴급조치(긴급조치 제9호)의 발령·적...,[다수의견] 보통 일반의 공무원을 표준으로 공무원이 직무를 집행하면서 객관적 주...,국가배상법 제2조 제1항$ 헌법 제29조$ 구 헌법(1980. 10. 27. 헌법 ...,대법원 2000. 5. 12. 선고 99다70600 판결(공2000하$ 1403)$...,【원고$ 상고인】 별지 원고 명단 기재와 같다. (소송대리인 법무법인 덕수 담당변호...
8,8,230901,법인세부과처분취소,2017두41313,20220825,판결,[1] 구 국세기본법 제14조 제3항을 적용하여 거래 등의 실질에 따라 과세하기...,[1] 구 국세기본법(2010. 1. 1. 법률 제9911호로 개정되기 전의 것...,[1] 구 국세기본법(2010. 1. 1. 법률 제9911호로 개정되기 전의 것)...,-,【원고$ 상고인】 주식회사 장미트레이딩 (소송대리인 법무법인(유한) 율촌 외 1인)...
9,9,230863,손해배상(기),2019다229202,20220825,전원합의체 판결,[1] 채권자대위권의 행사요건인 ‘보전의 필요성’을 인정하기 위한 요건 및 이를...,[1] 채권자는 자기의 채권을 보전하기 위하여 일신에 전속한 권리가 아닌 한 채...,[1] 민법 제404조 제1항 / [2] 민법 제404조 제1항,[1] 대법원 2020. 5. 21. 선고 2018다879 전원합의체 판결(공20...,【원고$ 피상고인】 삼성화재해상보험 주식회사 (소송대리인 법무법인 도원 외 1인)【...
